In [1]:
import h5py
import scipy.sparse as ss
import numpy as np


In [2]:
fname_10x = "/Users/fahimehb/Documents/VAE_GAN_project/dat/CTX_Hip_10x_counts.h5"
fname_ss = "/Users/fahimehb/Documents/VAE_GAN_project/dat/CTX_Hip_SS_counts.h5"

def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))

f_10x = h5py.File(fname_10x, mode='r')
f_10x.visititems(print_attrs)

print('------------------------')
f_ss = h5py.File(fname_ss, mode='r')
f_ss.visititems(print_attrs)


data
data/counts
data/gene
data/samples
data/shape
------------------------
data
data/exon
data/exon/dims
data/exon/i
data/exon/p
data/exon/x
data/intron
data/intron/dims
data/intron/i
data/intron/p
data/intron/x
data/t_exon
data/t_exon/dims
data/t_exon/i
data/t_exon/p
data/t_exon/x
data/t_intron
data/t_intron/dims
data/t_intron/i
data/t_intron/p
data/t_intron/x
data/total_exon_counts
data/total_intron_counts
gene_names
sample_names


# ss functions

In [39]:
def extract_sparse_mat(h5f, data_path):
    
    data = h5f[data_path]
    x = data['x']
    i = data['i']
    p = data['p']
    dims = data['dims']
    
    sparse_mat = ss.csc_matrix((x[0:x.len()],
                               i[0:i.len()],
                               p[0:p.len()]),
                               shape = (dims[0],dims[1]))
    return sparse_mat

def extract_gene_names(h5f, data_path):
    data = h5f[data_path]
    gene_names = data[:].astype(str).tolist()
    return gene_names


In [ ]:
h5f = h5py.File(fname_ss,'r')
exons = extract_sparse_mat(h5f,'/data/exon/')
introns = extract_sparse_mat(h5f,'/data/intron/')
X = exons + introns
X = X.todense()

In [45]:
gene_names = extract_gene_names(h5f, 'gene_names')
req_genes = ['Sst', 'Vip']

In [49]:
req_gene_indices = [gene_names.index(i) for i in req_genes]

In [51]:
X[:req_gene_indices]

TypeError: slice indices must be integers or None or have an __index__ method

# 10X functions

In [ ]:
def get_cellxgene_mat(h5f,gene_idx_sorted,n_samples, data_path):
    X = h5f[data_path][gene_idx_sorted, 0:n_samples]
    X = np.transpose(X)
    return X

def get_gene_names(h5f,gene_idx_sorted, gene_path):
    gene_list = h5f[gene_path][gene_idx_sorted].astype(str).tolist()
    return gene_list

In [20]:
# 10X

fname = fname_10x
data_path = 'data/counts/'
gene_path = 'data/gene/'

h5f = h5py.File(fname,'r')
n_samples = 10
gene_idx = np.array([0,1,2,3,4])
gene_idx_sorted = np.sort(gene_idx) #Required for slicing data in h5 file

X = get_cellxgene_mat(h5f,gene_idx_sorted,n_samples, data_path)
genes = get_gene_names(h5f, gene_idx_sorted, gene_path)

In [23]:
print(X)
print(genes)

[[ 8  0  0  0  0]
 [13  2  0  0  0]
 [ 8  0  0  0  0]
 [11  0  0  0  0]
 [10  1  0  0  0]
 [ 0  0  0  0  0]
 [ 5  0  0  0  0]
 [30  0  0  0  0]
 [17  0  0  0  0]
 [14  0  0  0  0]]
['Xkr4', 'Gm1992', 'Gm37381', 'Rp1', 'Sox17']
